## Data Analysis Simple Agent

### Overview

This tutorial guides you through creating an AI-powered data analysis agent that can interpret and answer questions about a dataset using natural language. It combines language models with data manipulation tools to enable intuitive data exploration.

### Motivation

Data analysis often requires specialized knowledge, limiting access to insights for non-technical users. By creating an AI agent that understands natural language queries, we can democratize data analysis, allowing anyone to extract valuable information from complex datasets without needing to know programming or statistical tools.

### Key Components

Language Model: Processes natural language queries and generates human-like responses
Data Manipulation Framework: Handles dataset operations and analysis
Agent Framework: Connects the language model with data manipulation tools
Synthetic Dataset: Represents real-world data for demonstration purposes
Method

Create a synthetic dataset representing car sales data
Construct an agent that combines the language model with data analysis capabilities
Implement a query processing function to handle natural language questions
Demonstrate the agent's abilities with example queries
Conclusion

This approach to data analysis offers significant benefits:

Accessibility for non-technical users
Flexibility in handling various query types
Efficient ad-hoc data exploration
By making data insights more accessible, this method has the potential to transform how organizations leverage their data for decision-making across various fields and industries.

In [5]:
# Important libraries
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents import AgentType
from langchain_openai import ChatOpenAI
import pandas as pd
import numpy as np

In [3]:
# import dataset
df = pd.read_csv("data/Car_sales.csv")
df.head()

,Manufacturer,Model,Sales in thousands,4-year resale value,Vehicle type,Price in thousands,Engine size,Horsepower,Wheelbase,Width,Length,Curb weight,Fuel capacity,Fuel efficiency,Latest Launch
0,Acura,Integra,16.919,16.36,Passenger,21.5,1.8,140,101.2,67.3,172.4,2.639,13.2,28,2-Feb-14
1,Acura,TL,39.384,19.875,Passenger,28.4,3.2,225,108.1,70.3,192.9,3.517,17.2,25,6-Mar-15
2,Acura,CL,14.114,18.225,Passenger,.,3.2,225,106.9,70.6,192,3.47,17.2,26,1-Apr-14
3,Acura,RL,8.588,29.725,Passenger,42,3.5,210,114.6,71.4,196.6,3.85,18,22,3-Oct-15
4,Audi,A4,20.397,22.255,Passenger,23.99,1.8,150,102.6,68.2,178,2.998,16.4,27,10-Aug-15


## Create data analysis agent

In [4]:
# initiate llm

llm = ChatOpenAI(
    model = 'gpt-4o',
    max_tokens = 1000,
    temperature=0
)

In [7]:
agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    allow_dangerous_code=True,
    agent_type=AgentType.OPENAI_FUNCTIONS   
)
print("Data Analysis Agent is ready. You can now ask questions about the data.")

Data Analysis Agent is ready. You can now ask questions about the data.


## Define Question-Asking Function

In [8]:
def ask_agent(question):
    """Function to ask questions to the agent and display the response"""
    response = agent.run({
        "input": question,
        "agent_scratchpad": f"Human: {question}\nAI: To answer this question, I need to use Python to analyze the dataframe. I'll use the python_repl_ast tool.\n\nAction: python_repl_ast\nAction Input: ",
    })
    print(f"Question: {question}")
    print(f"Answer: {response}")
    print("---")

In [10]:
# Example questions
ask_agent("What are the column names in this dataset?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.columns.tolist()'}`


['Manufacturer', 'Model', 'Sales in thousands', '4-year resale value', 'Vehicle type', 'Price in thousands', 'Engine size', 'Horsepower', 'Wheelbase', 'Width', 'Length', 'Curb weight', 'Fuel capacity', 'Fuel efficiency', 'Latest Launch']The column names in the dataset are:

1. Manufacturer
2. Model
3. Sales in thousands
4. 4-year resale value
5. Vehicle type
6. Price in thousands
7. Engine size
8. Horsepower
9. Wheelbase
10. Width
11. Length
12. Curb weight
13. Fuel capacity
14. Fuel efficiency
15. Latest Launch

> Finished chain.
Question: What are the column names in this dataset?
Answer: The column names in the dataset are:

1. Manufacturer
2. Model
3. Sales in thousands
4. 4-year resale value
5. Vehicle type
6. Price in thousands
7. Engine size
8. Horsepower
9. Wheelbase
10. Width
11. Length
12. Curb weight
13. Fuel capacity
14. Fuel efficiency
15. Latest Launch
---


In [13]:
ask_agent("What are the total number of cars by Acura?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "acura_cars_count = df[df['Manufacturer'] == 'Acura'].shape[0]\nacura_cars_count"}`


4There are a total of 4 cars by Acura in the dataframe.

> Finished chain.
Question: What are the total number of cars by Acura?
Answer: There are a total of 4 cars by Acura in the dataframe.
---


In [15]:
ask_agent("What is the model from Acura which has highest average mileage and how much is it")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Manufacturer'] == 'Acura'].sort_values(by='Fuel efficiency', ascending=False).iloc[0][['Model', 'Fuel efficiency']]"}`


Model              Integra          
Fuel efficiency                   28
Name: 0, dtype: objectThe Acura model with the highest average mileage is the Integra, with a fuel efficiency of 28 miles per gallon.

> Finished chain.
Question: What is the model from Acura which has highest average mileage and how much is it
Answer: The Acura model with the highest average mileage is the Integra, with a fuel efficiency of 28 miles per gallon.
---
